## FASE 1. Extracción de datos de películas

In [86]:
# importamos todas las librerias necesarias para realizar el ejercicio

import requests
import pandas as pd
import mysql.connector
from mysql.connector import Error
import numpy as np

In [87]:
# extraemos los datos de la API y la guardamos en una variable (en formato JSON)

url = "https://beta.adalab.es/resources/apis/pelis/pelis.json"

datos = requests.get(url)
datos.status_code
datos.content

datos_pelis = datos.json()

## FASE 2. Creación de la base de datos

In [89]:
# Iniciamos la connexion con SQL ---> Se deberá modificar el password acorde con el del usuario que ejecute el codigo. 
try:
    cnx = mysql.connector.connect(
        host='127.0.0.1',
        user='root',
        password='AlumnaAdalab',
    )
    print('Conexión exitosa')
except Error as e:
    print('Error al conectar:', e)


Conexión exitosa


In [90]:
# Creamos la base de datos 

try:
    mycursor = cnx.cursor()
    query = "CREATE DATABASE IF NOT EXISTS films_ejercicio_2"
    mycursor.execute(query)
    print("Query exitosa")
except:
    print("Error.")

Query exitosa


In [ ]:
# para crear las tablas primero debemos revisar los datos del API y analizar que columnas debemos crear.
datos_pelis

In [91]:
mycursor.execute("USE films_ejercicio_2")

mycursor.execute("DROP TABLE IF EXISTS films")

query = '''CREATE TABLE films (
    id INT PRIMARY KEY AUTO_INCREMENT,
    titulo VARCHAR(255) NOT NULL,
    año INT,
    duracion FLOAT,
    genero VARCHAR(255),
    adultos TINYINT(1),
    subtitulos TEXT
);'''

mycursor.execute(query)

## Fase 3. Inserción de los Datos en la Base de datos

In [92]:
#creamos un DF

df_films = pd.DataFrame(datos_pelis)
df_films

# Convertir DataFrame a lista de tuplas
#pd.notna(): Verifica si el valor no es NaN/null antes de insertarlo
#iterrows(): Itera sobre las filas del DataFrame

# Reemplazar NaN con None para que se inserten como NULL en la BD
# Reemplazar NaN con None de forma más robusta
df_films_clean = df_films.replace({np.nan: None, 'nan': None, 'NaN': None})


In [94]:
columnas = ['titulo','año','duracion','genero','adultos','subtitulos']

In [95]:
# Convertir DataFrame directamente a lista de tuplas
datos_films_clean = df_films_clean[columnas].values.tolist()  #selecciono las columnas de las que quiero hacer el insert, así no incorporo el ID



In [96]:
datos_films_clean

[['The Godfather', 1972, 175, 'Crimen', False, ['es', 'en']],
 ['The Godfather Part II', 1974, 202, 'Crimen', False, ['es', 'en']],
 ['Pulp Fiction', 1994, 154, 'Crimen', True, ['es', 'en']],
 ['Forrest Gump', 1994, 142, 'Drama', False, ['es', 'en', 'fr']],
 ['The Dark Knight', 2008, 152, 'Acción', False, ['es', 'en']],
 ['Fight Club', 1999, 139, 'Drama', True, ['es', 'en']],
 ['Inception', 2010, 148, 'Ciencia ficción', False, ['es', 'en', 'de']],
 ['The Matrix', 1999, 136, 'Ciencia ficción', False, ['es', 'en']],
 ['The Shawshank Redemption', 1994, 142, 'Drama', False, ['es', 'en']],
 ['Interstellar', 2014, 169, 'Ciencia ficción', False, ['es', 'en']],
 ['Gladiator', 2000, 155, 'Acción', False, ['es', 'en']],
 ['Titanic', 1997, 195, 'Romance', False, ['es', 'en', 'fr']],
 ['Avatar', 2009, 162, 'Ciencia ficción', False, ['es', 'en']],
 ['Saving Private Ryan', 1998, 169, 'Bélico', True, ['es', 'en']],
 ['The Silence of the Lambs', 1991, 118, 'Thriller', True, ['es', 'en']],
 ['Se7en', 1

In [97]:
# BOLEANOS. En SQL se representan con el TINYINT que es 1 o 0 , por lo que la columna de ADULTOS debemos cambiarla para que nos devuelva 0 o 1.
df_films_clean['adultos'] = df_films_clean['adultos'].apply(lambda x: int(x))

In [98]:
# para poder insertar los datos, debe estar en formato lista. En este momento, los datos contenidos en "subtitulos" es una lista. Deberiamos pasarlo a string
df_films_clean['subtitulos'] = df_films_clean['subtitulos'].apply(
    lambda x: ",".join(x) if isinstance(x, list) else x )

In [100]:
df_films_clean  #comprobamos que adultos nos devuelve 0 o 1 y que los subtitulos ya no es una lista (deberiamos crear una tabla con los idiomas en los subtitulos???)

,id,titulo,año,duracion,genero,adultos,subtitulos
0,1,The Godfather,1972,175,Crimen,0,"es,en"
1,2,The Godfather Part II,1974,202,Crimen,0,"es,en"
2,3,Pulp Fiction,1994,154,Crimen,1,"es,en"
3,4,Forrest Gump,1994,142,Drama,0,"es,en,fr"
4,5,The Dark Knight,2008,152,Acción,0,"es,en"
...,...,...,...,...,...,...,...
95,96,La vita è bella,1997,116,Drama,0,"es,en,it"
96,97,Requiem for a Dream,2000,102,Drama,1,"es,en"
97,98,Memento,2000,113,Thriller,1,"es,en"
98,99,Eternal Sunshine of the Spotless Mind,2004,108,Drama,0,"es,en"


In [101]:
datos_films = df_films_clean[columnas].values.tolist()

In [102]:
datos_films

[['The Godfather', 1972, 175, 'Crimen', 0, 'es,en'],
 ['The Godfather Part II', 1974, 202, 'Crimen', 0, 'es,en'],
 ['Pulp Fiction', 1994, 154, 'Crimen', 1, 'es,en'],
 ['Forrest Gump', 1994, 142, 'Drama', 0, 'es,en,fr'],
 ['The Dark Knight', 2008, 152, 'Acción', 0, 'es,en'],
 ['Fight Club', 1999, 139, 'Drama', 1, 'es,en'],
 ['Inception', 2010, 148, 'Ciencia ficción', 0, 'es,en,de'],
 ['The Matrix', 1999, 136, 'Ciencia ficción', 0, 'es,en'],
 ['The Shawshank Redemption', 1994, 142, 'Drama', 0, 'es,en'],
 ['Interstellar', 2014, 169, 'Ciencia ficción', 0, 'es,en'],
 ['Gladiator', 2000, 155, 'Acción', 0, 'es,en'],
 ['Titanic', 1997, 195, 'Romance', 0, 'es,en,fr'],
 ['Avatar', 2009, 162, 'Ciencia ficción', 0, 'es,en'],
 ['Saving Private Ryan', 1998, 169, 'Bélico', 1, 'es,en'],
 ['The Silence of the Lambs', 1991, 118, 'Thriller', 1, 'es,en'],
 ['Se7en', 1995, 127, 'Thriller', 1, 'es,en'],
 ['Goodfellas', 1990, 146, 'Crimen', 1, 'es,en'],
 ['The Departed', 2006, 151, 'Crimen', 1, 'es,en'],
 ['

In [103]:
# Query de inserción
query_insert = """
INSERT INTO films (titulo, año, duracion, genero, adultos, subtitulos)
                              
VALUES (%s, %s, %s, %s, %s, %s)
"""

In [104]:
# Ejecutar inserción
mycursor.executemany(query_insert, datos_films)
cnx.commit()

print(f"{mycursor.rowcount} registros insertados")

print(f"{mycursor.rowcount} registros insertados")

100 registros insertados
100 registros insertados


## FASE 4. Obtener información a partir de los datos.